# Load dataset

In [11]:
load_file_name = 'dataset/labeled-requirements.txt'
seed = 7

#####################################################
import pickle
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
ann_info = {0: '/component/', 1: '/refinement_of_component/', 2: '/action/', 
            3: '/refinement_of_action/',
            4: '/condition/', 5: '/priority/', 6: '/motivation/', 7: '/role/',
            8: '/object/', 9: '/refinement_of_object/',
            10: '/sub_action/', 11: '/sub_argument_of_action/', 12: '/sub_priority/', 
            13: '/sub_role/', 14: '/sub_object/',
            15: '/sub_refinement_of_object/', 16: '/none/'}


####################
""" Load Dataset """
####################
# flip key, value pairs in the dictionary
ann_info = dict((v,k) for k,v in ann_info.items())

X = []
Y = []
currentX = []
currentY = []
split_sequences = True

for line in open(load_file_name):
    line = line.rstrip()

    if line:
        row = line.split()
        word, tag = row
        currentX.append(word)
        currentY.append(ann_info[tag])
    
    elif split_sequences: # the end of sentence
        X.append(currentX)
        Y.append(currentY)
        currentX = []
        currentY = []

print("The total number of sentences:", len(X))

##############################
""" Shuffle Data with seed """
##############################
# shuffle...
merged_data = list(zip(X, Y))
random.seed(seed)
random.shuffle(merged_data)

X, Y = zip(*merged_data)
X = list(X)
Y = list(Y)
assert(len(X) == len(Y))

The total number of sentences: 704


In [12]:
# key-value switching
ann_info = {val:key for (key, val) in ann_info.items()}

def tkn_assign(y, boolean_list):
    boolean_list[y] = True
def increment_cnt(sent_num_list, boolean_list):
    for i in range(0, len(sent_num_list)):
        if boolean_list[i] == True:
            sent_num_list[i] += 1
            boolean_list[i] = False

boolean_list = [False] * 17
sent_num_list = [0] * 17

for i, sent in enumerate(Y):
    for j, token in enumerate(sent):
        tkn_assign(Y[i][j], boolean_list)
    increment_cnt(sent_num_list, boolean_list)    
    
print('# of sentences per class')
for i in range(0, len(sent_num_list)):
    print(sent_num_list[i], '\t', ann_info[i])

# of sentences per class
308 	 /component/
32 	 /refinement_of_component/
687 	 /action/
269 	 /refinement_of_action/
129 	 /condition/
664 	 /priority/
81 	 /motivation/
300 	 /role/
653 	 /object/
202 	 /refinement_of_object/
117 	 /sub_action/
45 	 /sub_argument_of_action/
49 	 /sub_priority/
51 	 /sub_role/
101 	 /sub_object/
30 	 /sub_refinement_of_object/
703 	 /none/


# Data Cleaning

정말 기초적인 단계에서 데이터를 손질하자. 형태적인 변형은 여기서 하지 않는다. (엄격한 기준으로) 삭제하거나 추가하기만 한다. 물론 의도적으로 마음에 안드는 (부분적으로) 데이터를 삭제해서는 안되고 삭제하려면 form형태와 같이 삭제되어야 되는 기준이 있어야 한다. (unseen data에 대한 대비를 해야되기 때문이다.)
  1. 정말 말도 안되거나 진짜 도움 안되는 것(noise)들만 삭제: etc. 의 .삭제
  2. 모든 문장 끝에 콤마 있도록 하기
  
보통 puntuation은 NLP에서 삭제하기 마련인데, 여기 task에서 자주 등장하는 puntuation은 유용하게 사용될 수 있다. 물론 매우 드물게 등장하는 것들은 infrequend token으로 분류되어 삭제하는 것이 generalization하는게 도움을 준다. <br>
분류되는 단위가 token이기 때문에 그들의 위치정보도 많이 도움이 될 것이다. <br>
다른 것들은 아직 잘 판단이 안서나, terminator term인 콤마는 유용하게 사용될 수 있다. 왜냐면 그들의 label은 모두 none이면서 동시에 항상 문장 맨 끝에 위치하기 때문이다. 따라서, noise가 아닌 좋은 signal로 작용할 수 있다.

## data cleaning for parsing doing well 

In [13]:
##### Parser가 잘 동작하기 위해서 (parsing에 방해되는) 불필요한 charactor들은 삭제한다. 
##### Parser가 이상하게 동작하면 모델에 치명적 손상을 입힌다. 따라서 최대한 이쁘게 input으로 넣어주는게 필요하다.

#""" Punctuation """
# 마지막 콤마빼고는 왠만한 punctuation은 다 지운다.

### 하이픈 (-) 관련
# 동사,명사 중에 end-user와 같이 -하이픈이 있는 경우, 왼쪽 char들과 함께 삭제한다.
# 단, non- 패턴은 하이픈만 삭제한다. e.g. non-clinical -> nonclinical 
# 단, pre- 패턴은 하이픈만 삭제한다. e.g. pre-paid -> prepaid
# 단, re- 패턴은 하이픈만 삭제한다. e.g. re-enter -> reenter
# 단, un- 
# 단, -up 패턴은 하이픈만 삭제한다. e.g. pop-up -> popup
# 단, -in 패턴은 하이픈만 삭제한다. e.g. mail-in -> mailin
### 아니 그냥 여기선 다 삭제

# 단, -based, -coded 패턴들은 하이픈과 based, coded를 삭제한다.
# 단, e-mail, check-box인 경우도 하이픈만 삭제
# 단, 고유명사인경우 하이픈만 삭제 e.g. FACT-ADDRESS -> FACTADDRESS
# 2개의 하이픈이 있으면 뒤에 있는 하이픈을 기준으로 한다. e.g. side-by-side - > side


### 예외 명사들 (e.g. ????-??-??)
# 명사, 동사에 따라 duck으로 바꾼다. (그냥 duck이 명사,동사 다 해당하는 단어라서...)

### 오퍼스트로피 (') 관련
# 문법적인 '는 제외한다.
# ' ' 쌍이 있는 경우에 button, option, funtion, feature, item, section 등과 같이 옆에 관련 명사가 있으면 다음과 같이 처리한다. soft-require text 특징인데, button, option 등을 보조 설명하는 말이다. parsing에 방해된다.
# 오른쪽에 명사가 있는 경우 ' '-> beutiful 치환하고 왼쪽에 명사가 있으면 그냥 다 삭제
# 단, ' '쌍 안에 명사가 하나면 그냥 '만 삭제한다.
# 오른쪽 왼쪽 명사가 모두 없으면 또는 왼쪽에 형용사가 있고 오른쪽에 명사가 없을경우: feature로 치환해준다.
# 동사구에서 맨 오른쪽이 명사이면 명사를 살려둔다. 

### 정상적이지 않은 패턴들 관련
# 중간 중간에 ... .와 같이 이어져있는 패턴들 -> 지운다 또는 ... 여러개 있는거 삭제 또는 한개로
# 맨 앞에 or이 있으면 삭제

### 부연 설명하는 i.e. 와 e.g. 관련
# 부연 설명하는 i.e와 e.g.는 annotation하지 않아도 큰 문제가 되지 않는다.
# 따라서, 괄호로 둘러싸여있는 i.e. 와 e.g.가 있을 경우 모두 삭제한다. (,로 구분되어 있는 경우도 삭제를 하였다.)
# 단, 괄호가 없으면 i.e.또는 e.g.만 삭제한다.

### and/or 관련
# 정확한 parsing을 하기 위해 모두 and로 통일한다.

#""" for a clear sentence """
## Parser가 잘 동작하기 위해서 문장이 1개가 있어야 한다.
## 따라서 input data는 하나의 문장이 이루워져있다고 가정한다.
## 단, 하나의 문장은 여러개의 clause들을 포함한다. 그리고 여러개의 clause들을 and/or로 확장한 것도 하나의 문장이라고 간주한다.

### :와 함께 보충 설명하는 문장 삭제
# "This is a feature request: It would be nice to have a Rename checkbox on the encrypt menu along with the feature and beutiful checkboxes"
# -> "It would be nice to have a Rename checkbox on the encrypt menu along with the feature and beutiful checkboxes"
# "May I propose that: Results are shown one per line.
# -> Results are shown one per line.


### : -> where로 바꿔준다. (우리의 데이터에서 :은 문장들을 연결해주는 연결고리이다.)
# "The display shall have two regions:   left 2/3 of the display is graphical, right 1/3 of the display is a data table."
# 여기서 :를 where로 교체한다. (하나의 문장으로 바꾸기 위해)

### : -> which are (접속사)로 바꿔준다. 
# "The table side of the display shall be split into 2 regions:  sequential and temporal."
# "The table side of the display shall be split into 2 regions which are sequential and temporal."

### So는 독립적인 2개의 문장을 뜻한다. 하나로 만들어주기 위해 so that구문으로 바꾼다.
# "As an administrator I want to have centralised configuration so I can remotely change settings across all units"
# "As an administrator I want to have centralised configuration so that I can remotely change settings across all units"

### ; -> which are
# The confirmation must contain the following information; the dispute case number, the type of chargeback requested (pending or immediate) and the date that the merchant response is due.
# The confirmation must contain the following information which are the dispute case number, the type of chargeback requested (pending or immediate) and the date that the merchant response is due.

### ; -> so that
# "It would be nice to add a column to mailbox table; this would be used to store a path to user sieve script"
# "It would be nice to add a column to mailbox table so that this would be used to store a path to user sieve script"

### 다음과 같이 괄호 삭제
#  by (1) the dispute case number, (2) the merchant account number, (3) the cardmember account number and (4) the issuer number

##### 불필요한 표현들 삭제 (특히 앞부분에서)
## Hello, Also, Hi, In general, In addtion, Or, i think 등등..
## e.g., In addition to the above criteria -> 삭제
## getSeries (); -> (); 삭제

##### it would be X if 패턴
##### if -> that으로 바꿔준다. 
# 보통 if절은 부사절이다. 하지만, 이 패턴이 있는 문장에서 if절은 주절로 사용된다. 
# 따라서, if를 that로 바꿔 parser가 주절로 인식할 수 있도록 해준다.

##### 명령문이 있을 경우
# 맨앞에 it을 그냥 추가
# e.g. allow option to hide tray icon -> it allow option to hide tray icon

### change word
for i, sentence in enumerate(X):
    for j, token in enumerate(sentence):
        if token == 'maybe':
            X[i][j] = 'may'
            
### 자동사 전치사 삭제
# search for 와 같은 자동사의 전치사를 삭제해줘서 object가 제대로 인식하도록 해준다.
# # Convert tuple to list
# X = list(X)
# Y = list(Y)

# etc. 를 tokenization하면 'etc', '.'가 된다.
# etc. -> terminator .와 구별되야 한다.
# 따라서 etc 다음의 .는 그냥 삭제하도록 한다.
cnt = 0
for i, sentence in enumerate(X):
    for j, token in enumerate(sentence):
        if sentence[j] == '.':
            if sentence[j-1] == 'etc':
                sentence.remove(sentence[j])  # X delete
                del Y[i][j]
                cnt += 1
print('DELETE counts = ', cnt)

puntation_list = ['(', ')', ';']
# # 이상하게도 한 번만 for loop 돌면서 삭제하면, 완전히 다 삭제가 안된다.
# # 또 다른 이유가 있겠지만, 그냥 여기서 for loop를 2번 돌려서 삭제한다.

# First delete
cnt = 0
for i, sentence in enumerate(X):
    for j, token in enumerate(sentence):
        if any(token in t for t in puntation_list):
            sentence.remove(sentence[j])  # X delete
            del Y[i][j]
            cnt += 1
print('(first) DELETE puntuation counts = ', cnt)

# Second delete
cnt = 0
for i, sentence in enumerate(X):
    for j, token in enumerate(sentence):
        if any(token in t for t in puntation_list):
            sentence.remove(sentence[j])  # X delete
            del Y[i][j]
            cnt += 1
print('(second) DELETE puntuation counts = ', cnt)

# terminator comma add!
cnt = 0 
for i, sentence in enumerate(X):
    if not sentence[-1] == '.':
        if not sentence[-1] == '?':
            X[i] = X[i] + ['.'] # 조심: X[i] += ['.'] 사용x 
            Y[i] = Y[i] + [16] # none label 
            cnt += 1
print('ADD counts = ', cnt)

DELETE counts =  3
(first) DELETE puntuation counts =  108
(second) DELETE puntuation counts =  2
ADD counts =  149


## Split train/test data

In [14]:
validation_size = 0.20 
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=validation_size, random_state=seed)
print('train:', len(X_train), ',  test:',len(X_test))

train: 563 ,  test: 141


## Write output files

In [15]:
### Write out files
def dump(data, name):
    filehandler = open(name,"wb")
    pickle.dump(data, filehandler)
    filehandler.close()
    
dump(X_train, 'X_train')
dump(Y_train, 'Y_train')
dump(X_test, 'X_test')
dump(Y_test, 'Y_test')